In [147]:
"""
function gatherInfo() {
    var result = {
        'gpx': '',
        'title': document.title,
        'marker': {
        }
    }
    var photos = {}
    $("a.fapp").each(function(i, v) {
        var text = $(v).find("p").text()
        var href = $(v).attr("href")
        if (text) {
            if (text == "針山陡峭山徑") {
                text = "針山的陡峭山徑"
            }
            if (!photos[text]) {
                photos[text] = []
            }
            photos[text].push(href)
        }
    })
    for (var i in map._layers) {
        var layer = map._layers[i];
        var gpx = layer._gpx
        if (gpx) {
            result["gpx"] = gpx
        }
        var icon = layer._icon
        if (icon) {
            if (!result.marker[icon.src]) {
                result.marker[icon.src] = []  
            }
            var latlng = layer._latlng
            var tmp = document.createElement("div")
            tmp.innerHTML = layer._popup._content
            var name = tmp.innerText
            var href = tmp.firstChild.getAttribute('href')
            var item = {"latlng": latlng, "name": name}
            if (href) {
                item['href'] = href
            }
            if (name !== "加水站") {
                var fixMap = {
                    "遠眺白沙灣，橋咀，東壩及蚺蛇尖": "遠眺白沙灣,橋咀,東壩及蚺蛇尖",
                    "眺望西貢西群山及馬鞍山": "眺望馬鞍山及西貢西群山",
                    "針山日落": "針山觀日落",
                    "針山頂眺望沙田": "針山遠眺沙田",
                    "遠眺吐露港及馬鞍山": "草山遠眺吐露港及馬鞍山",
                    "草山頂日落": "草山望大帽山日落",
                    "雨後溪流": "溪流",
                    "大牛湖頂 (2)": "大牛湖頂",
                    "大牛湖頂 (3)": "大牛湖頂"
                }
                if (fixMap[name]) {
                    name = fixMap[name]
                }
                var images = photos[name]
                if (images) {
                    var image = images[0]
                    item["image"] = image
                    if (images.length > 1) {
                        photos[name] = images.slice(1)
                    } else {
                        delete(photos[name])
                    }
                }
            }
            result.marker[icon.src].push(item)
        }
        
    }
    console.log(JSON.stringify(photos))
    return result;
}

var result = gatherInfo();
console.log(JSON.stringify(result))
var next = $("a.view-more.gtm-download").last()
if (next.text() == "下一段") {
    window.location.href = next.attr("href")
}


"""

# 麥理浩徑, 衛奕信徑, 鳳凰徑, 港島徑
# https://www.hiking.gov.hk/trail/info/id/VFFUbFFNOXNXTHNWU2FqT09UTjlOZz09
# https://www.hiking.gov.hk/trail/info/id/TCtocVVTL1JuMXpaQVBSOFBYYzRSUT09
# https://www.hiking.gov.hk/trail/info/id/YUNwM2czOWpZU2hrTXF3NlRTU3hNZz09
# https://www.hiking.gov.hk/trail/info/id/MkpYc3Jud1ZjS2RFbzE1Qkhza0htdz09


TRACK_NAME = "港岛径"
JSON_FILE = TRACK_NAME + ".json"



import json
import xml.etree.ElementTree as ET
from functools import reduce
from urllib.request import urlretrieve
import os
import shutil
from zipfile import ZipFile 
from math import radians, cos, sin, asin, sqrt

WORKING_DIR = './temp'
# create work dirf
if not os.path.exists(WORKING_DIR):
    os.mkdir(WORKING_DIR)
CACHE_FILE_DIR = './temp/files'
if not os.path.exists(CACHE_FILE_DIR):
    os.mkdir(CACHE_FILE_DIR)


# distance in kilometers
# https://www.geeksforgeeks.org/program-distance-two-points-earth/
def geodistance(lat1, lon1, lat2, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a)) 
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
      
    # calculate the result
    return(c * r)

# https://zhuanlan.zhihu.com/p/408786267
def footpoint(lat, lon, lat0, lon0, lat1, lon1):
    dx = lat1 - lat0;
    dy = lon1 - lon0;
    cross = dx * (lat - lat0) + dy * (lon - lon0)
    d2 = dx * dx + dy * dy
    u = cross / d2
    return (lat0 + u * dx, lon0 + u * dy)

def sortbydistance(markers, lat0, lon0, lat1, lon1):
    for marker in markers:
        pt = footpoint(marker['latlng']['lat'], marker['latlng']['lng'], lat0, lon0, lat1, lon1)
        marker['sort'] = pt[0] * (1 if lat0 < lat1 else -1)
    return sorted(markers, key=lambda x: x['sort'])

# for n tracks
titleList = []    # string * n
coordsList = []   # string of '<gx:coord>' * n
iconList = []     # url of icon * l, l is number of different icons
markersList = []  # {latlng, name, index of icon} * m * n, m is number of markers in each track


with open(JSON_FILE, 'r') as f:
    tracks = json.loads(f.read())

for trackObj in tracks:
    title = trackObj['title']
    title = title.replace('郊野樂行 - ', '')
    titleList.append(title)

    root = ET.fromstring(trackObj['gpx'])
    ns = {'ns': 'http://www.topografix.com/GPX/1/0'}

    pts = root.findall('./ns:trk/ns:trkseg/ns:trkpt', ns)
    # 麦理浩径第二段，轨迹方向反向
    if title.find("麥理浩徑") >= 0 and title.find("第二段") >= 0 or title.find("衛奕信徑") >= 0 and title.find("第四段") >= 0:
        pts = list(reversed(pts))
    coord = list(map(lambda pt: pt.get("lon") + "," + pt.get("lat") + "," + pt[0].text, pts))
    coords = " ".join(coord)
    coordsList.append(coords)
    
    isfenghuang = title.find("鳳凰徑") >= 0
    markers = []
    for icon in trackObj['marker']:
        if not icon in iconList:
            iconList.append(icon)
        iconIndex = iconList.index(icon)
        items = trackObj['marker'][icon] # list
        for j, item in enumerate(items):
            # Fix wrong coordinate in 凤凰径
            if isfenghuang:
                if item['latlng']['lat'] > 23:
                    item['latlng']['lat'] -= 1
                    item['latlng']['lng'] -= 1
                    # 羗山山頂
                    if item['latlng']['lat'] > 23:
                        item['latlng']['lat'] -= 1
                        item['latlng']['lng'] -= 1
                elif item['latlng']['lat'] < 22:
                    item['latlng']['lat'] += 1
                    item['latlng']['lng'] += 1
            
            lat = item['latlng']['lat']
            lng = item['latlng']['lng']
            
            # 过滤重复的
#             if j > 0:
#                 pre = items[j - 1]
#                 if pre['name'] == item['name']:
#                     if abs(lat - pre['latlng']['lat']) < 0.001:
#                         continue
#                     else:
#                         print(item['name'], title)
            item['icon'] = iconIndex
            minIndex = 0
            minDistance = 10000000
            for m, pt in enumerate(pts):
                dist = geodistance(lat, lng, float(pt.get("lat")), float(pt.get("lon")))
                if dist < minDistance:
                    minIndex = m
                    minDistance = dist
#             print(minDistance, minIndex, item['name'], dist)
            item['index'] = minIndex
            markers.append(item)
    
    markers = sorted(markers, key=lambda x: x['index'])
    if len(pts) > 1:
        sameList = list(filter(lambda x: x['index'] == 0, markers))
        if len(sameList) > 0:
            lat0 = float(pts[0].get("lat"))
            lon0 = float(pts[0].get("lon"))
            lat1 = float(pts[1].get("lat"))
            lon1 = float(pts[1].get("lon"))
            newSameList = sortbydistance(sameList, lat0, lon0, lat1, lon1)
            markers[0:len(newSameList)] = newSameList
        
        sameList = list(filter(lambda x: x['index'] == len(pts) - 1, markers))
        if len(sameList) > 0:
            lat0 = float(pts[len(pts) - 2].get("lat"))
            lon0 = float(pts[len(pts) - 2].get("lon"))
            lat1 = float(pts[len(pts) - 1].get("lat"))
            lon1 = float(pts[len(pts) - 1].get("lon"))
            newSameList = sortbydistance(sameList, lat0, lon0, lat1, lon1)
            markers[-len(newSameList):] = newSameList

    markersList.append(markers)
    

def export(coordsList, styles, markersList, name):
    # 重新创建临时目录
    
    trackNode = ""
    for i, coords in enumerate(coordsList):
        trackNode += """
                <Placemark>
                    <name><![CDATA[""" + titleList[i] + """]]></name>
                    <styleUrl>#LineStringStyle""" + str(i % 2 + 1) + """</styleUrl>
                    <LineString>
                        <coordinates>""" + coords + """</coordinates>
                    </LineString>
                </Placemark>"""

    styles = reduce(lambda x, y: x + y, ["""
            <Style id="marker_""" + str(i) + """\">
                <IconStyle>
                    <Icon>
                        <href>""" + k + """</href>
                    </Icon>
                </IconStyle>
            </Style>""" for i, k in enumerate(iconList)], "")

    
    markers = ""
    filenames = []
    for i, k in enumerate(markersList):
        for j, item in enumerate(k):
            description = ""
            if 'image' in item:
                image = item['image']
                filename = image[image.rindex('/')+1:]
                realpath = CACHE_FILE_DIR + os.sep + filename
                if not os.path.exists(realpath):
                    urlretrieve("https:" + image, realpath)
                filenames.append(filename)
                
                description = """
                    <description><![CDATA[<div><img src=\"files/""" + filename + """\"/></div>]]></description>"""
            elif 'href' in item:
                description = """
                    <description><![CDATA[<a href=\"""" + item['href'] + """\">""" + item['name'] + """</a>]]></description>"""
            markers += """
                <Placemark>
                    <name><![CDATA[""" + item['name'] + """]]></name>""" + description + """
                    <styleUrl>#marker_""" + str(item['icon']) +  """</styleUrl>
                    <Point>
                        <coordinates>""" + str(item['latlng']['lng']) + "," + str(item['latlng']['lat']) + """</coordinates>
                    </Point>
                </Placemark>"""


    # color: AABBGGRR
    gpx = """<?xml version="1.0" encoding="UTF-8"?>
    <kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">
        <Document id="TbuluKmlVersion2">
            <name><![CDATA[""" + name[name.find(os.sep)+1:] + """]]></name>
            <description><![CDATA[信息整理自郊野乐行 https://www.hiking.gov.hk]]></description>
            <Style id="LineStringStyle1">
                <LineStyle>
                    <color>ff0000ff</color>
                    <width>6.000000</width>
                </LineStyle>
            </Style>
            <Style id="LineStringStyle2">
                <LineStyle>
                    <color>ffff0000</color>
                    <width>6.000000</width>
                </LineStyle>
            </Style>""" + styles + """
            <Folder id="TbuluHisPointFolder">
                <name><![CDATA[标注]]></name>""" + markers + """
            </Folder>
            <Folder>
                <name><![CDATA[轨迹]]></name>""" + trackNode + """
            </Folder>
        </Document>
    </kml>"""

    with open(WORKING_DIR + '/doc.kml', 'w') as f:
        f.write(gpx)
    with ZipFile(name + ".kmz", 'w') as zipfile:
        zipfile.write(WORKING_DIR + '/doc.kml', 'doc.kml')
        for filename in filenames:
            zipfile.write(CACHE_FILE_DIR + os.sep + filename, "files/" + filename)

trackName = TRACK_NAME
if not os.path.exists(trackName):
    os.mkdir(trackName)

for seg in range(len(coordsList)):
    export(coordsList[seg:seg+1], styles, markersList[seg:seg+1], trackName + os.sep + trackName + "分段" + str(seg + 1))

export(coordsList, styles, markersList, trackName + os.sep + trackName + "全部")

print('Finished!')


Finished!
